In [1]:
import os
!pip install pyspark
!pip install findspark


# Start a SparkSession
import findspark
findspark.init()
import pandas as pd

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
#creating a datafram of the URL using Sparkfiles
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [5]:
# 2. Create a temporary view of the DataFrame called home_sales
home_sales_df.createOrReplaceTempView('home_sales')

In [6]:
#show header of first 5 rows of temporary view home_sales
spark.sql("select * from home_sales limit 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places
#filter to rows with "bedrooms" = 4
home_sales_3_df = home_sales_df.filter(home_sales_df["bedrooms"] == 4) #Assign filtered DataFrame to home_sales_3_df
home_sales_3_df.show() # Show the results of the filtered DataFrame
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places
#Extract first 4 characters of "date" and alias column as "year"
#Extract first 4 characters of "date_built" and alias column as "year_built"
from pyspark.sql.functions import year
from pyspark.sql.types import StringType, IntegerType
# Cast the "date_built" column to StringType before applying the year function
home_sales_3_df = home_sales_3_df.withColumn("date_str", home_sales_3_df["date"].cast(StringType())) #Keep home_sales_3_df as a Spark DataFrame
home_sales_3_df = home_sales_3_df.withColumn("year", year(home_sales_3_df["date_str"]))
#Show unique values of year within home_sales_df
home_sales_3_df.select("year").distinct().show()


#Calculate mean of price rounded to 2 decimals, grouping by "year"
from pyspark.sql.functions import mean, round
avg_price_per_year = home_sales_3_df.groupBy("year").agg(round(mean("price"), 2).alias("mean_price"))
print(avg_price_per_year.show())

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|f233cb41-6f33-4b0...|2021-07-18|      2016|437375|       4|        3|       1704|   11721|     2|         0|  34|
|2dc662fe-57b8-44b...|2020-04-20|      2016|417215|       4|        2|       2104|    8227|     2|         0|  48|
|d5f2062a-8950-417...|2019-09-02|      2016|305980|       4|        2|       1799|   13004|     1|         0|  26|
|f5e01433-f7b0-44e...|2021-03-10|      2010|335326|       4|        3|       1588|   14107|     1|         0|   8|
|a818abe9-fec6-48d...|2019-08-20|      2017|334284|       4|        2|       237

In [8]:
# 4. What is the average price of a home for each year the home was built,that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
#filter to rows with "bedrooms" = 3 and "bathrooms" == 3
# Corrected: Assign the filtered DataFrame to home_sales_4_df instead of calling show()
home_sales_4_df = home_sales_df.filter((home_sales_df["bedrooms"] == 3) & (home_sales_df["bathrooms"] == 3))
home_sales_4_df.show()
#Extract first 4 characters of "date_built" and alias column as "year_built"
from pyspark.sql.functions import year
from pyspark.sql.types import StringType, IntegerType
# Cast the "date_built" column to StringType before applying the year function
home_sales_4_df = home_sales_4_df.withColumn("date_built_str", home_sales_4_df["date_built"].cast(StringType()))
home_sales_4_df = home_sales_4_df.withColumn("year_built", year(home_sales_4_df["date_built_str"]))


#Show unique values of year within home_sales_df
home_sales_4_df.select("year_built").distinct().show()


#Calculate mean of price rounded to 2 decimals, grouping by "year"
from pyspark.sql.functions import mean, round
avg_price_per_year_built = home_sales_4_df.groupBy("year_built").agg(round(mean("price"), 2).alias("mean_price"))
print(avg_price_per_year_built.show())

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|
|2ed8d509-7372-46d...|2021-08-06|      2015|258710|       3|        3|       1918|    9666|     1|         0|  25|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|
|941bad30-eb49-4a7...|2020-05-09|      2015|229896|       3|        3|       2197|    8641|     1|         0|   3|
|ea620c7b-c2f7-4c6...|2021-05-31|      2011|437958|       3|        3|       2356|   11052|     1|         0|  26|
|0cfe57f3-28c2-472...|2019-10-04|      2015|308313|       3|        3|       196

In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?
#filter to rows with "bedrooms" = 3 and "bathrooms" == 3 and "floors" == 2 $gte 2,000 Sqr Ft
#Assign the filtered DataFrame to home_sales_5_df and remove .show()
home_sales_5_df = home_sales_df.filter((home_sales_df["bedrooms"] == 3) & (home_sales_df["bathrooms"] == 3) & (home_sales_df["floors"] == 2) & (home_sales_df["sqft_living"] >= 2000))

#Extract first 4 characters of "date_built" and alias column as "year_built"
from pyspark.sql.functions import year
from pyspark.sql.types import StringType, IntegerType
# Cast the "date_built" column to StringType before applying the year function
#Use home_sales_5_df instead of home_sales_df here
home_sales_5_df = home_sales_5_df.withColumn("date_built_str", home_sales_5_df["date_built"].cast(StringType()))
#Use home_sales_5_df instead of home_sales_df here
home_sales_5_df = home_sales_5_df.withColumn("year_built", year(home_sales_5_df["date_built_str"]))

#Show unique values of year within home_sales_df
home_sales_5_df.select("year_built").distinct().show()

#Calculate mean of price rounded to 2 decimals, grouping by "year"
from pyspark.sql.functions import mean, round
avg_price_per_year_built = home_sales_5_df.groupBy("year_built").agg(round(mean("price"), 2).alias("mean_price"))
print(avg_price_per_year_built.show())

+----------+
|year_built|
+----------+
|      2015|
|      2013|
|      2014|
|      2012|
|      2016|
|      2010|
|      2011|
|      2017|
+----------+

+----------+----------+
|year_built|mean_price|
+----------+----------+
|      2015| 297609.97|
|      2013| 303676.79|
|      2014| 298264.72|
|      2012| 307539.97|
|      2016|  293965.1|
|      2010| 285010.22|
|      2011| 276553.81|
|      2017| 280317.58|
+----------+----------+

None


In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places, having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
#Filter home_sales_df for for price greater or equal to $350,000
home_sales_6_df = home_sales_df.filter(home_sales_df["price"] >= 350000)

#aggregate mean price of homes in home_sales_df by "view", sorting in descending order of 'view'
from pyspark.sql.functions import mean, round
avg_price_per_view = home_sales_6_df.groupBy("view").agg(round(mean("price"), 2).alias("mean_price"))
avg_price_per_view.sort(avg_price_per_view["view"].desc()).show()


#calculate start time and end time of calculation in seconds
start_time = time.time()
end_time = time.time() - start_time
print("--- %s seconds ---" % (end_time))


+----+----------+
|view|mean_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 9.751319885253906e-05 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')



In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')
#assign cached home_sales to new df
home_sales_cache_df = spark.table("home_sales")


In [13]:
# 9. Using the cached data, run the last query above, that calculates the average price of a home per "view" rating, rounded to two decimal places, having an average home price greater than or equal to $350,000.
#Run avg_price_per_view using the filters of home_sales_6_df on the chached home_sales table
#Filter home_sales_df for for price greater or equal to $350,000
home_sales_6cache_df = home_sales_cache_df.filter(home_sales_cache_df["price"] >= 350000)

from pyspark.sql.functions import mean, round
avg_price_per_view = home_sales_6cache_df.groupBy("view").agg(round(mean("price"), 2).alias("mean_price"))
avg_price_per_view.sort(avg_price_per_view["view"].desc()).show()

# Determine the runtime and compare it to the uncached runtime.
#run avg_price_per_view using cacheTabe('home_sales')

start_time = time.time()
end_time = time.time() - start_time
print("--- %s seconds ---" % (end_time))


+----+----------+
|view|mean_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 8.726119995117188e-05 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")



In [15]:
# 11. Read the parquet formatted data.
home_sales_partitioned_df = spark.read.parquet("home_sales_partitioned")


In [16]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned_df.createOrReplaceTempView('home_sales_partitioned')
home_sales_partitioned_df = spark.table("home_sales_partitioned")


In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
home_sales_6part_df = home_sales_partitioned_df.filter(home_sales_partitioned_df["price"] >= 350000)

from pyspark.sql.functions import mean, round
avg_price_per_view = home_sales_6part_df.groupBy("view").agg(round(mean("price"), 2).alias("mean_price"))
avg_price_per_view.sort(avg_price_per_view["view"].desc()).show()

print("--- %s seconds ---" % (end_time))

+----+----------+
|view|mean_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 8.726119995117188e-05 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')



False